In [2]:
import geopandas as gpd
from pymapd import connect
import timeit
import os

In [3]:
MAPD_STORAGE = %env MAPD_STORAGE
MAPD_ERROR_LOG = os.path.join(MAPD_STORAGE,'data/mapd_log/mapd_server.ERROR')

In [5]:
pwd = %pwd
shapefile = 'building_footprint_polys_w_ids.shp'
shapepath = os.path.join(pwd, shapefile)
print('Shape path is: {}'.format(shapepath))

Shape path is: /home/mapdadmin/demo/tahoe/laz/building_footprint_polys_w_ids.shp


In [6]:
local_bld = gpd.read_file(shapefile)
local_bld.head()

,PAS,LU_ID,TMDL_CODE,TMDL_DESC,LU_DESC,IMP_ID,COVER_ID,IMP_DESC,COVER_DESC,Impervious,Bld_ID2,ID,geometry
0,Yes,1003,0.0,None,Trails-Impervious,4,1,Building,Hard,1.0,1,1,POLYGON ((-13367379.78247419 4703625.392935817...
1,Yes,1003,0.0,None,Roads_Impervious,4,1,Building,Hard,NaN,2,2,POLYGON ((-13366397.43129916 4704896.538210514...
2,No,1003,0.0,None,Trails-Impervious,4,1,Building,Hard,1.0,3,3,POLYGON ((-13369135.87060145 4703817.955506972...
3,Yes,1003,0.0,None,Roads_Impervious,4,1,Building,Hard,NaN,4,4,POLYGON ((-13366462.57526186 4704994.943857891...
4,Yes,1003,0.0,None,Roads_Impervious,4,1,Building,Hard,NaN,5,5,"POLYGON ((-13366414.7051146 4705007.353525128,..."


In [7]:
start_time = timeit.default_timer()
try:
    con = connect(user="mapd", password="HyperInteractive!", host="localhost", dbname="mapd")
except:
    print('Problem connecting to local mapd instance, check config')
print('Connecting to mapd took {0:.2f} seconds'.format(timeit.default_timer() - start_time))

Connecting to mapd took 0.24 seconds


In [9]:
# import from web mercator shapefile using geo option
table_name = 'tahoe_bld_footprints'
query = "COPY {} FROM '{}' ".format(table_name, shapepath)
query += "WITH (GEO='True');"
query

"COPY tahoe_bld_footprints FROM '/home/mapdadmin/demo/tahoe/laz/building_footprint_polys_w_ids.shp' WITH (GEO='True');"

In [10]:
try:
    con.execute(query)
except:
    print('oops')

In [51]:
mapd_tables_list = con.get_tables()

In [53]:
for t in mapd_tables_list:
    if t == table_name:
        print(t)

tahoe_bld_footprints


In [57]:
# table exists
# why does this get key error 16?
con.get_table_details(table_name)

KeyError: 16

In [44]:
def print_char_numbers(input_str):
    upper_str = ''
    lower_str = ''
    for s in range(0,len(input_str)):
        upper_str + str(s)[0]
        lower_str += str(s % 10)
    print(upper_str)
    print(lower_str)

In [58]:
#10.76391 * ST_AREA(mapd_geo) AS sqft_mapd
query = 'SELECT Bld_ID2, '
query += 'ST_AREA(mapd_geo) AS sqm_mapd '
query += 'FROM {} ;'.format(table_name)
print_char_numbers(query)
query


0123456789012345678901234567890123456789012345678901234567890123456789012


'SELECT Bld_ID2, ST_AREA(mapd_geo) AS sqm_mapd FROM tahoe_bld_footprints ;'

In [59]:
try:
    con.execute(query)
except:
    print('oops')

In [63]:
try:
    df = con.select_ipc(query)
    df.head()
except:
    print('Problem running query')
    !tail -50 {MAPD_ERROR_LOG}

In [64]:
df

,Bld_ID2,sqm_mapd
0,1001,2.602519e-09
1,1002,2.767138e-09
2,1003,2.147317e-09
3,1004,7.419658e-09
4,1005,1.964963e-09
5,1006,1.806256e-09
6,1007,1.734406e-09
7,1008,6.446044e-09
8,1009,1.359967e-08
9,1010,1.306944e-09


In [65]:
df.mean()

Bld_ID2     2.510450e+04
sqm_mapd    1.570475e-08
dtype: float64